In [1]:
## import libraries
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# date and time utilities from the standard library
from datetime import datetime, date, timedelta
from sklearn.metrics import mean_absolute_error
from lightgbm import LGBMRegressor
# library to suppress warnings for cleaner output
import warnings
warnings.filterwarnings('ignore')

# custom library 
# import enefit

In [2]:
# update data directory path
DATA_DIR = '/kaggle/input/predict-energy-behavior-of-prosumers'

In [3]:
## get the full path of all files
# walk through the every directory inside DATA_DIR 
for dirpath, dirnames, filenames in os.walk(DATA_DIR):
    # within each directory, iterate over every filename
    for filename in filenames:
        # join the directory path and filename
        print(os.path.join(dirpath, filename))

/kaggle/input/predict-energy-behavior-of-prosumers/client.csv
/kaggle/input/predict-energy-behavior-of-prosumers/gas_prices.csv
/kaggle/input/predict-energy-behavior-of-prosumers/electricity_prices.csv
/kaggle/input/predict-energy-behavior-of-prosumers/weather_station_to_county_mapping.csv
/kaggle/input/predict-energy-behavior-of-prosumers/public_timeseries_testing_util.py
/kaggle/input/predict-energy-behavior-of-prosumers/historical_weather.csv
/kaggle/input/predict-energy-behavior-of-prosumers/county_id_to_name_map.json
/kaggle/input/predict-energy-behavior-of-prosumers/train.csv
/kaggle/input/predict-energy-behavior-of-prosumers/forecast_weather.csv
/kaggle/input/predict-energy-behavior-of-prosumers/example_test_files/sample_submission.csv
/kaggle/input/predict-energy-behavior-of-prosumers/example_test_files/client.csv
/kaggle/input/predict-energy-behavior-of-prosumers/example_test_files/gas_prices.csv
/kaggle/input/predict-energy-behavior-of-prosumers/example_test_files/electricity

In [4]:
# read the CSV files into DataFrames
train = pd.read_csv(os.path.join(DATA_DIR, "train.csv"))
gas_df = pd.read_csv(os.path.join(DATA_DIR, "gas_prices.csv"))
electricity_df = pd.read_csv(os.path.join(DATA_DIR, "electricity_prices.csv"))
client_df = pd.read_csv(os.path.join(DATA_DIR, "client.csv"))
fw_df = pd.read_csv(os.path.join(DATA_DIR, "forecast_weather.csv"))
hw_df = pd.read_csv(os.path.join(DATA_DIR, "historical_weather.csv"))

# read a file from a different directory
# locations = pd.read_csv("/kaggle/input/fabiendaniels-mapping-locations-and-county-codes/county_lon_lats.csv")

In [5]:
train.head()

,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id
0,0,0,1,0.713,0,2021-09-01 00:00:00,0,0,0
1,0,0,1,96.590,1,2021-09-01 00:00:00,0,1,0
2,0,0,2,0.000,0,2021-09-01 00:00:00,0,2,1
3,0,0,2,17.314,1,2021-09-01 00:00:00,0,3,1
4,0,0,3,2.904,0,2021-09-01 00:00:00,0,4,2


In [6]:
train.describe().T

,count,mean,std,min,25%,50%,75%,max
county,2018352.0,7.297034e+00,4.780990,0.0,3.000,7.000,1.100000e+01,15.000
is_business,2018352.0,5.368261e-01,0.498642,0.0,0.000,1.000,1.000000e+00,1.000
product_type,2018352.0,1.898927e+00,1.081766,0.0,1.000,2.000,3.000000e+00,3.000
target,2017824.0,2.748556e+02,909.502378,0.0,0.378,31.133,1.802062e+02,15480.274
is_consumption,2018352.0,5.000000e-01,0.500000,0.0,0.000,0.500,1.000000e+00,1.000
data_block_id,2018352.0,3.218746e+02,182.634314,0.0,166.000,323.000,4.790000e+02,637.000
row_id,2018352.0,1.009176e+06,582648.179597,0.0,504587.750,1009175.500,1.513763e+06,2018351.000
prediction_unit_id,2018352.0,3.304538e+01,19.590594,0.0,16.000,33.000,5.000000e+01,68.000


In [7]:
gas_df.head()

,forecast_date,lowest_price_per_mwh,highest_price_per_mwh,origin_date,data_block_id
0,2021-09-01,45.23,46.32,2021-08-31,1
1,2021-09-02,45.62,46.29,2021-09-01,2
2,2021-09-03,45.85,46.40,2021-09-02,3
3,2021-09-04,46.30,46.80,2021-09-03,4
4,2021-09-05,46.30,46.58,2021-09-04,5


Preprocessing Data

In [8]:
''''
# drop missing values of target 
data.dropna(subset=['target'], inplace=True)

# convert datetime to UTC
data['datetime'] = pd.to_datetime(data['datetime'], utc=True)

# rename 'forecast_date' column to 'datetime' for consistency before merging
electricity = electricity.rename(columns={'forecast_date': 'datetime'}) 

# convert datetime to UTC
electricity['datetime'] = pd.to_datetime(electricity['datetime'], utc=True)

# reduce 'block_id' in the 'client' DataFrame by 2 to match 'train' DataFrame
client['data_block_id'] -= 2

# remove the 'Unnamed: 0' column from the 'locations' DataFrame
locations.drop('Unnamed: 0', axis=1, inplace=True)

# round 'latitude' and 'longitude' to 1 decimal place
'''


"'\n# drop missing values of target \ndata.dropna(subset=['target'], inplace=True)\n\n# convert datetime to UTC\ndata['datetime'] = pd.to_datetime(data['datetime'], utc=True)\n\n# rename 'forecast_date' column to 'datetime' for consistency before merging\nelectricity = electricity.rename(columns={'forecast_date': 'datetime'}) \n\n# convert datetime to UTC\nelectricity['datetime'] = pd.to_datetime(electricity['datetime'], utc=True)\n\n# reduce 'block_id' in the 'client' DataFrame by 2 to match 'train' DataFrame\nclient['data_block_id'] -= 2\n\n# remove the 'Unnamed: 0' column from the 'locations' DataFrame\nlocations.drop('Unnamed: 0', axis=1, inplace=True)\n\n# round 'latitude' and 'longitude' to 1 decimal place\n"

In [9]:
# def preTrain(data, client, histweather)